In [1]:
from keras.models import Sequential
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, LSTM
from keras.utils import np_utils

import numpy as np

In [2]:
digit = '0123456789'
alpha = 'abcdefghij'

In [3]:
char_set = list(set(digit + alpha)) # id -> char
char_dic = {w: i for i, w in enumerate(char_set)}

In [4]:
data_dim = len(char_set) # one hot encoding size
seq_length = time_steps = 7
num_classes = len(char_set)

In [5]:
# Build training date set
dataX = []
dataY = []

for i in range(1000):
  rand_pick = np.random.choice(10, 7)
  x = [char_dic[digit[c]] for c in rand_pick]
  y = [char_dic[alpha[c]] for c in rand_pick]
  dataX.append(x)
  dataY.append(y)

In [6]:
# One-hot encoding
dataX = np_utils.to_categorical(dataX, num_classes = num_classes)
# reshape X to be [samples, time steps, features]
dataX = np.reshape(dataX, (-1, seq_length, data_dim))

In [7]:
# One-hot encoding
dataY = np_utils.to_categorical(dataY, num_classes = num_classes)
# time steps
dataY = np.reshape(dataY, (-1, seq_length, data_dim))

In [8]:
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE
# note: in a situation where your input sequences have a variable length,
# use input_shape=(None, nb_feature).
model.add(LSTM(32, input_shape = (time_steps, data_dim), return_sequences = False))

# For the decoder's input, we repeat the encoded input for each time step
model.add(RepeatVector(time_steps))
# The decoder RNN could be multiple layers stacked or a single layer

model.add(LSTM(32, return_sequences = True))

# For each of step of the output sequence, decide which character should
# be chosen
model.add(TimeDistributed(Dense(data_dim)))
model.add(Activation('softmax'))

In [9]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [10]:
model.fit(dataX, dataY, epochs = 1000)

Epoch 1/1000
32/32 [==============================] - 9s 15ms/step - loss: 2.9019 - accuracy: 0.1156
Epoch 2/1000
32/32 [==============================] - 1s 16ms/step - loss: 2.4539 - accuracy: 0.1193
Epoch 3/1000
32/32 [==============================] - 0s 11ms/step - loss: 2.2716 - accuracy: 0.1687
Epoch 4/1000
32/32 [==============================] - 0s 12ms/step - loss: 2.1868 - accuracy: 0.2139
Epoch 5/1000
32/32 [==============================] - 0s 10ms/step - loss: 2.1273 - accuracy: 0.2357
Epoch 6/1000
32/32 [==============================] - 0s 10ms/step - loss: 2.0820 - accuracy: 0.2599
Epoch 7/1000
32/32 [==============================] - 0s 9ms/step - loss: 2.0387 - accuracy: 0.2660
Epoch 8/1000
32/32 [==============================] - 0s 11ms/step - loss: 2.0031 - accuracy: 0.2774
Epoch 9/1000
32/32 [==============================] - 0s 12ms/step - loss: 1.9647 - accuracy: 0.2811
Epoch 10/1000
32/32 [==============================] - 0s 12ms/step - loss: 1.9356 - accurac

In [11]:
# Create test data set for fun
testX = []
testY = []

for i in range(10):
  rand_pick = np.random.choice(10, 7)
  x = [char_dic[digit[c]] for c in rand_pick]
  y = [alpha[c] for c in rand_pick]
  testX.append(x)
  testY.append(y)

In [12]:
# One-hot encoding
testX = np_utils.to_categorical(testX, num_classes = num_classes)
# reshape X to be [samples, time steps, features]
testX = np.reshape(testX, (-1, seq_length, data_dim))

In [13]:
predictions = model.predict(testX, verbose = 0)

for i, prediction in enumerate(predictions):
  x_index = np.argmax(testX[i], axis = 1)
  x_str = [char_set[j] for j in x_index]

  index = np.argmax(prediction, axis = 1)
  result = [char_set[j] for j in index]

  print(''.join(x_str), ' -> ', ''.join(result), " true: ", ''.join(testY[i]))

0646493  ->  ageeejg  true:  agegejd
3409243  ->  deaccdd  true:  deajced
2414779  ->  ceeeehg  true:  cebehhj
0455476  ->  aafffhh  true:  aeffehg
9660440  ->  jggeaae  true:  jggaeea
8884786  ->  iiihiig  true:  iiiehig
0864439  ->  aigeejj  true:  aigeedj
0090398  ->  aajjjid  true:  aajadji
0265226  ->  accfggd  true:  acgfccg
5570878  ->  ffhaaia  true:  ffhaihi
